<a href="https://colab.research.google.com/github/gopalprashad81/HybridSeleniumFramework/blob/main/MachineLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================
# Core Libraries
# ============================
import numpy as np
import pandas as pd

# ============================
# Scikit-Learn (sklearn)
# ============================
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ============================
# LightGBM
# ============================
import lightgbm as lgb

# ============================
# Visualization
# ============================
import matplotlib.pyplot as plt
import seaborn as sns

# ============================
# Optional: Suppress Warnings
# ============================
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# lecture 3
# rmse_train=metrics.root_mean_squared_error(y_true=train['y'],y_pred=old.predict(train.drop(columns
from sklearn.metrics import median_absolute_error
msdae_train = median_absolute_error(y_true=train['y'], y_pred=pls.predict(train.drop(columns=['y'])))
msdae_test =median_absolbute_error(x_test, x_pred_test)                                                                                     )))

In [ ]:
import sklearn.linear_model

In [ ]:
huber=  HuberRegressor().fit(X_train,y_train)
y_pred_huber  = huber.predict(x_test)
# compare fits on test sample
rmse_ols =mean_squared_error(y_test,y_pred_ols)
rmse_huber = mean_sqaured_error(y_test,y_pred_huber,squared=False)

print("rmse_ols", rmse_ols)
print("rmse_huber",rmse_huber)

In [8]:
# ============================================================
# Problem Set 3 — Elastic Net Prediction of Monthly Stock Returns
# Author: Gopal Prashad
# ============================================================
import pandas as pd
import numpy as np
import dask.dataframe as dd  # Use Dask for out-of-core processing to handle large data
from dask import delayed, compute
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

try:
    from memory_profiler import profile
except ImportError:
    print("memory_profiler not installed. Install via 'pip install memory_profiler' for usage tracking.")


    def profile(func):  # Dummy decorator if not installed
        return func


# ============================================================
# LOAD AND CLEAN DATA (Hints #2, #6) - Use Dask for memory efficiency
# ============================================================
data_dir = '/content/Problem3/

# Load with Dask (lazy, partitioned for large files)
df = dd.read_parquet(data_dir + 'OpenAP_Macro.parquet.gzip', engine='pyarrow')
print("Loaded dataset lazily with Dask.")

# Standardize column names to lowercase (compute meta)
df.columns = [str(c).strip().lower() for c in df.columns]

# Print columns and head to identify names (compute small sample)
print("Dataframe columns after loading:")
print(df.columns.tolist())

print("\nDataframe head:")
print(df.head())

# Convert date to datetime and align to month-end (Hint #2) - Dask supports vectorized ops
df['date'] = dd.to_datetime(df['dateym']) + pd.offsets.MonthEnd(0)

# Add year column for filtering
df['year'] = df['date'].dt.year

# Filter for years 2000 and afterwards (lazy filter)
df = df[df['year'] >= 2000]
print(f"Filtered to years >= 2000: {df['date'].min().compute()} to {df['date'].max().compute()}")


# Downcast in Dask (meta propagation for dtypes)
def downcast_df_partition(df_partition):
    float_cols = df_partition.select_dtypes(include=['float64']).columns
    df_partition[float_cols] = df_partition[float_cols].astype('float32')
    int_cols = df_partition.select_dtypes(include=['int64']).columns
    df_partition[int_cols] = df_partition[int_cols].apply(pd.to_numeric, downcast='integer')
    return df_partition


# Apply downcast per partition (lazy)
df = df.map_partitions(downcast_df_partition, meta=df)

print("Downcasted dtypes for memory efficiency.")

# ============================================================
# IDENTIFY KEY COLUMNS AND VERIFY INTEGRITY (Data Hygiene)
# ============================================================

# Identify firm identifier (permno), date, and return column dynamically
permno_col = next((c for c in ['permno', 'id'] if c in df.columns), None)
assert permno_col is not None, "No firm identifier (permno/id) found."

ret_col = next((c for c in ['retadj', 'return', 'ret_excess'] if c in df.columns), None)
assert ret_col is not None, " No return column found."

# Verify no missing keys (compute to check)
assert (~df[permno_col].isna()).all().compute() and (~df['date'].isna()).all().compute(), "Missing firm IDs or dates."

# Check for duplicate keys (group and count, compute max)
duplicates = df.groupby([permno_col, 'date']).size().max().compute()
assert duplicates == 1, f"Duplicates found: max group size {duplicates}"

print(" Primary key (permno, date) uniqueness confirmed.")


# ============================================================
# FEATURE IDENTIFICATION AND PREPROCESSING (Hints #3, #4)
# ============================================================
# Exclude non-features
exclude_cols = {permno_col, 'date', 'year', ret_col}
macro_vars = ['macro_dp', 'macro_ep', 'macro_bm', 'macro_ntis', 'macro_tbl', 'macro_tms', 'macro_dfy', 'macro_svar']  # Goyal macros without slashes
# Assert macro vars exist
missing_macros = [v for v in macro_vars if v not in df.columns]
assert not missing_macros, f"Missing macro variables: {missing_macros}. Check column names."

firm_vars = [col for col in df.columns if col not in exclude_cols and col not in macro_vars and 'float' in str(df[col].dtype) or 'int' in str(df[col].dtype)]

# Fill missing firm vars with period median (Hint #4, Gu et al. 2020) - Dask groupby.transform
for col in firm_vars:
    df[col] = df.groupby('date')[col].transform(lambda x: x.fillna(x.median()), meta=(col, 'f8'))

# Fill any remaining NaNs in firm_vars with 0 (neutral value after ranking)
for col in firm_vars:
    df[col] = df[col].fillna(0)

# Fill missing macro vars with overall mean (compute mean first)
for col in macro_vars:
    mean_val = df[col].mean().compute()
    df[col] = df[col].fillna(mean_val)

# Identify binary firm vars (compute unique per col - expensive, so sample or compute on subset if large)
binary_vars = []
non_binary_firm = []
for col in firm_vars:
    uniques = df[col].dropna().unique().compute()
    if set(uniques) <= {0, 1}:
        binary_vars.append(col)
    else:
        non_binary_firm.append(col)

# Preprocess: rank non-binary to [-1,1], map binary to {-1,1} (Hint #3) - Custom partition func for ranking (Dask doesn't support rank(pct=True) directly)
def preprocess_partition(partition):
    """Partition-wise preprocessing: Rank continuous features to [-1,1] interval for outlier robustness and scale invariance."""
    partition = partition.reset_index(drop=True)  # For local ranking
    for col in non_binary_firm:
        ranks = partition[col].rank(pct=True)
        partition[col] = 2 * ranks - 1
    for col in binary_vars:
        partition[col] = 2 * partition[col] - 1
    return partition

# Apply per partition (approximates global per-date, but for exact per-date ranking, compute groupby - may need .compute() if too large)
# Note: For exact cross-sectional ranking, better to groupby 'date' but Dask groupby.apply is shuffle-heavy; approx with partitions if dates sorted
df = df.map_partitions(preprocess_partition, meta=df)

# Assert no remaining NaNs in predictors (compute check)
predictors = firm_vars + macro_vars  # No interactions between macro and firm vars
assert not df[predictors].isna().any().any().compute(), "NaNs still present in predictors after imputation."

print(f" Preprocessed {len(firm_vars)} firm vars and {len(macro_vars)} macro vars.")


# ============================================================
# ROLLING WINDOW SETUP AND MODEL FITTING (Hints #5, #7, #10, #11)
# ============================================================
start_year = 2000
initial_train_years = 6
val_years = 3

# Get unique years for annual retraining (compute)
years = sorted(df['year'].unique().compute())
test_years = [y for y in years if y >= start_year + initial_train_years + val_years]


@profile  # Memory profile the function if profiler installed
def process_year(test_year, df, predictors, ret_col):
    """Process a single test year: Train model annually, predict for all months in the year using monthly-updated predictors."""
    train_end = test_year - val_years - 1
    val_start = train_end + 1
    val_end = test_year - 1

    # Create separate training and validation DataFrames (lazy)
    train_df = df[(df['year'] >= start_year) & (df['year'] <= train_end)]  # Training set
    val_df = df[(df['year'] >= val_start) & (df['year'] <= val_end)]  # Validation set
    test_df = df[df['year'] == test_year]  # Test set (all months in year)

    # Compute subsets to memory for sklearn (only load needed data)
    train_df = train_df.compute()
    val_df = val_df.compute()
    test_df = test_df.compute()

    # Combine in memory for fitting
    train_val_df = pd.concat([train_df, val_df])
    test_fold = np.concatenate([-1 * np.ones(len(train_df)), np.zeros(len(val_df))])  # -1: train, 0: val

    # Create PredefinedSplit CV object for validation separation
    ps = PredefinedSplit(test_fold)

    X_tv = train_val_df[predictors]
    y_tv = train_val_df[ret_col]

    # ElasticNetCV: Tune alpha and l1_ratio with predefined CV split, parallelize with n_jobs
    enet = ElasticNetCV(alphas=[1, 10], l1_ratio=[0.5, 0.9], cv=ps, n_jobs=-1, random_state=42)
    enet.fit(X_tv, y_tv)  # Fit model with training/validation separation

    # Predict for all months in test year using monthly-updated predictors
    X_test = test_df[predictors]
    y_pred = enet.predict(X_test)

    test_df['pred'] = y_pred
    return test_df[['date', permno_col, ret_col, 'pred']]


# Parallelize processing across test years using Dask
delayed_tasks = [delayed(process_year)(test_year, df, predictors, ret_col) for test_year in test_years]
predictions = compute(*delayed_tasks)  # Compute all in parallel; auto-uses multi-cores

all_pred = pd.concat(predictions)


# ============================================================
# EVALUATION METRICS (Hint #9)
# ============================================================
def calculate_r2_oos(y_true, y_pred):
    """Calculate out-of-sample R² as per Gu et al. (2020) Equation (19): 1 - SSE / total sum of squares (no mean subtraction)."""
    sse = np.sum((y_true - y_pred) ** 2)
    total_ss = np.sum(y_true ** 2)
    return (1 - sse / total_ss) * 100  # As percentage


y_true = all_pred[ret_col]
y_pred = all_pred['pred']

r2_oos = calculate_r2_oos(y_true, y_pred)

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
medae = median_absolute_error(y_true, y_pred)

print("Elastic Net Results:")
print(f"R²_oos (Gu et al. Eq. 19): {r2_oos:.2f}%")
print(f"RMSE: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Median Absolute Error: {medae:.4f}")

# Save predictions (Hint #12)
all_pred.to_csv(data_dir + 'predictions.csv', index=False)
print(" Predictions saved to " + data_dir + "predictions.csv")

SyntaxError: unterminated string literal (detected at line 26) (ipython-input-2443794992.py, line 26)